In [28]:
import json
import glob
import random
import os

from scipy import stats
from sklearn.metrics import confusion_matrix, classification_report

from eval import metric_utils

In [2]:


# sampled = []
# for f in glob.glob("longfact/longfact-objects_gpt4_01-12-2024_noduplicates/*.jsonl"):
#     data = [json.loads(line.strip()) for line in open(f)]
#     sampled.extend(random.choices(data, k=3))

# directory = "longfact/object_sampled_114"
# os.makedirs(directory, exist_ok=True)

# with open(f"{directory}/114.jsonl", "w") as fw:
#     for sample in sampled:
#         fw.write(f"{json.dumps(sample)}\n")

In [3]:
with open("results/2024-05-16-11-15-06-correlation_vs_factscore.json") as jsonfile:
    colbert = json.load(jsonfile)

with open("results/2024-05-16-12-21-08-correlation_vs_factscore.json") as jsonfile:
    serper = json.load(jsonfile)

with open("results/2024-05-28-00-00-55-SAFE.json") as jsonfile:
    longfact_1 = json.load(jsonfile)

with open("results/2024-05-27-23-15-41-SAFE.json") as jsonfile:
    longfact_2 = json.load(jsonfile)

In [4]:
idx = 3
for i in range(len(serper['rate_facts']["per_prompt_data"][idx]["checked_statements"])):
    s = serper['rate_facts']["per_prompt_data"][idx]["checked_statements"][i]
    c = colbert['rate_facts']["per_prompt_data"][idx]["checked_statements"][i]
    if s["annotation"] != c["annotation"]:
        for f in serper['rate_facts']["per_prompt_data"][idx]["sentence_to_atomic_fact_and_label"][s["sentence"]]:
            if f["atomic_fact"] == s["atomic_fact"]:
                print(f["label"])
        print(s["self_contained_atomic_fact"])
        print(c["self_contained_atomic_fact"])
        print()
        if s["rate_data"] is not None:
            print(s["rate_data"]["response"])
        print("---")
        if c["rate_data"] is not None:
            print(c["rate_data"]["response"])
        print("=====")

S
Malcolm Hedding was born in South Africa.
Malcolm Hedding was born in South Africa.

1. Malcolm Hedding was born in 1952 into a British descent family that had settled in the Eastern Cape region.
2. He experienced a visitation of Jesus in the garden of his home situated within a gold-mining community near Johannesburg.
3. He is a white South African by birth who was born in 1952 when the Afrikaner Nationalist Party was in power.

Based on the provided knowledge, it is strongly implied that Malcolm Hedding was indeed born in South Africa.

STATEMENT: Malcolm Hedding was born in South Africa.
[Supported]
---
1. Malcolm Hedding was born in 1952 into a British descent family in the Eastern Cape region.
2. He entered the ministry in the early 1970s as a member of the Assemblies of God of Southern Africa.
3. Hedding confronted the apartheid system in South Africa from the pulpit and was forced to flee his homeland.
4. Between 1986 and 1989, he was a pastor in Jerusalem and served as the Ch

In [5]:
serper_s = [prompt["Supported"] for prompt in serper['rate_facts']["per_prompt_data"]]
serper_ns = [prompt["Not Supported"] for prompt in serper['rate_facts']["per_prompt_data"]]

colbert_s = [prompt["Supported"] for prompt in colbert['rate_facts']["per_prompt_data"]]
colbert_ns = [prompt["Not Supported"] for prompt in colbert['rate_facts']["per_prompt_data"]]

fs_s = [prompt["metrics"]["S"] for prompt in serper['rate_facts']["per_prompt_data"]]
fs_ns = [prompt["metrics"]["NS"] for prompt in serper['rate_facts']["per_prompt_data"]]

print(stats.spearmanr(serper_s, fs_s))
print(stats.spearmanr(colbert_s, fs_s))
print(stats.spearmanr(serper_s, colbert_s))
print(stats.spearmanr(serper_ns, fs_ns))
print(stats.spearmanr(colbert_ns, fs_ns))
print(stats.spearmanr(serper_ns, colbert_ns))

SignificanceResult(statistic=0.7948224937151211, pvalue=8.107113550868753e-05)
SignificanceResult(statistic=0.7115508058574904, pvalue=0.0009276247988976871)
SignificanceResult(statistic=0.932987138874981, pvalue=1.6549801216133154e-08)
SignificanceResult(statistic=0.6542056074766355, pvalue=0.003225062631569224)
SignificanceResult(statistic=0.5836801490313335, pvalue=0.01098913657724503)
SignificanceResult(statistic=0.826923523694436, pvalue=2.31588976284608e-05)


In [6]:
serper_preds = []
colbert_preds = []
fs_labels = []

d = {
    "Supported": "S",
    "Not Supported": "NS",
    "Irrelevant": "IR"
}

for idx in range(18):
    for i in range(len(serper['rate_facts']["per_prompt_data"][idx]["checked_statements"])):
        s = serper['rate_facts']["per_prompt_data"][idx]["checked_statements"][i]
        c = colbert['rate_facts']["per_prompt_data"][idx]["checked_statements"][i]

        for f in serper['rate_facts']["per_prompt_data"][idx]["sentence_to_atomic_fact_and_label"][s["sentence"]]:
            if f["atomic_fact"] == s["atomic_fact"]:
                fs_labels.append(f["label"])
                break
        serper_preds.append(d[s["annotation"]])
        colbert_preds.append(d[c["annotation"]])

In [7]:
key = "Not Supported"
lf1 = [prompt["side2_posthoc_eval_data"][key] for prompt in longfact_1["per_prompt_data"]]
lf2 = [prompt["side2_posthoc_eval_data"][key] for prompt in longfact_2["per_prompt_data"]]

print(stats.spearmanr(lf1, lf2))

KeyError: 'side2_posthoc_eval_data'

In [8]:
with open("results/gemma2bsft-lf-response-f1_0.85-object_38-SAFE.json") as jsonfile:
    lf = json.load(jsonfile)

l = ["side2_posthoc_eval_data" in prompt for prompt in lf["per_prompt_data"]]
print(sum(l), len(l))

35 38


In [9]:
lf["per_prompt_data"][0]["side2_posthoc_eval_data"]["Not Supported"]

0

In [12]:
print(sum([s == f for s, f in zip(serper_preds, fs_labels)]))
print(sum([c == f for c, f in zip(colbert_preds, fs_labels)]))
print(sum([c == s for c, s in zip(colbert_preds, serper_preds)]))

411
427
539


In [13]:
print(classification_report(fs_labels, serper_preds))
print()
print(classification_report(fs_labels, colbert_preds))

              precision    recall  f1-score   support

          IR       0.00      0.00      0.00        84
          NS       0.67      0.41      0.51       229
           S       0.62      0.92      0.74       344

    accuracy                           0.63       657
   macro avg       0.43      0.44      0.42       657
weighted avg       0.56      0.63      0.57       657


              precision    recall  f1-score   support

          IR       0.00      0.00      0.00        84
          NS       0.68      0.48      0.56       229
           S       0.64      0.92      0.76       344

    accuracy                           0.65       657
   macro avg       0.44      0.47      0.44       657
weighted avg       0.57      0.65      0.59       657



In [2]:

from common import modeling
from common import shared_config
from common import utils
from eval import metric_utils
from eval.safe import config as safe_config
from eval.safe import search_augmented_factuality_eval as safe

In [4]:
rater_model = modeling.Model(
  safe_config.model,
  temperature=safe_config.model_temp,
  max_tokens=safe_config.max_tokens,
)

In [3]:
from datasets import load_dataset

In [5]:
from eval.safe import classify_relevance
from eval.safe import get_atomic_facts
from eval.safe import rate_atomic_fact


In [247]:

with open("results/gemma2bsft-object_38-SAFE-shard0.json") as jsonfile:
    lf1 = json.load(jsonfile)

lf1 = lf1["per_prompt_data"]

with open("results/gemma2bsft-object_38-SAFE-shard1.json") as jsonfile:
    lf1.extend(json.load(jsonfile)["per_prompt_data"])

with open("results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-object_38-SAFE.json") as jsonfile:
    lf2 = json.load(jsonfile)

lf2 = lf2["per_prompt_data"]

# with open("results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-object_38-SAFE-shard1.json") as jsonfile:
#     lf2.extend(json.load(jsonfile)["per_prompt_data"])


In [270]:
l = [19] # range(38)
for i in l:
    prompt = lf1[i]["prompt"]
    lf1_eval = lf1[i]["side2_posthoc_eval_data"]
    for j in range(38):
        lf2_eval = lf2[j]["side2_posthoc_eval_data"]
        if lf2[j]["prompt"] == prompt:
            print((lf1_eval["Supported"], lf1_eval["num_claims"]), (lf2_eval["Supported"], lf2_eval["num_claims"]))
            print()
            print("prompt", prompt)
            print("---")
            print(lf1[i]["side2_response"])
            for statement in lf1[i]["side2_posthoc_eval_data"]["checked_statements"]:
                print('###')
                print(statement["self_contained_atomic_fact"])
                print(statement["annotation"])
            print("===")
            print(lf2[j]["side2_response"])
            for statement in lf2[j]["side2_posthoc_eval_data"]["checked_statements"]:
                print('###')
                print(statement["self_contained_atomic_fact"])
                print(statement["annotation"])
            break
         

(37, 67) (56, 65)
prompt Could you provide some details about the Hague Conventions? Provide as many specific details and examples as possible (such as names of people, numbers, events, locations, dates, times, etc.)
---
Sure! The Hague Conventions are a set of international treaties that were designed to protect the rights of civilians in international armed conflicts. Here are some specific details and examples related to the Hague Conventions:

1. The First Hague Convention: The First Hague Convention was signed in 1899 and was designed to protect the rights of civilians in international armed conflicts. It was signed by 31 countries and established the principle that civilians should not be targeted by military forces.

Example: In 1904, during the Boxer Rebellion in China, British and French troops were sent to suppress the rebellion. However, they were ordered to avoid killing or injuring the Chinese civilians. The British and French troops complied with this principle and did no

In [139]:
prompts = {}
for i in range(19):
    prompt = lf1['per_prompt_data'][i]["prompt"]
    prompts[prompt] = {"Supported": [0, 0], "Not Supported": [0, 0], "num_claims": [0, 0]}

    prompt = lf1_0['per_prompt_data'][i]["prompt"]
    prompts[prompt] = {"Supported": [0, 0], "Not Supported": [0, 0], "num_claims": [0, 0]}

for i in range(19):
    for key in ["Supported", "Not Supported", "num_claims"]:
        prompts[lf1['per_prompt_data'][i]["prompt"]][key][0] = lf1['per_prompt_data'][i]['side2_posthoc_eval_data'][key]
        prompts[lf1_0['per_prompt_data'][i]["prompt"]][key][0] = lf1_0['per_prompt_data'][i]['side2_posthoc_eval_data'][key]
        prompts[lf2['per_prompt_data'][i]["prompt"]][key][1] = lf2['per_prompt_data'][i]['side2_posthoc_eval_data'][key]
        prompts[lf2_0['per_prompt_data'][i]["prompt"]][key][1] = lf2_0['per_prompt_data'][i]['side2_posthoc_eval_data'][key]

for prompt, values in prompts.items():
    print(values)
        

{'Supported': [166, 70], 'Not Supported': [1, 8], 'num_claims': [168, 106]}
{'Supported': [72, 83], 'Not Supported': [5, 24], 'num_claims': [82, 119]}
{'Supported': [87, 53], 'Not Supported': [24, 19], 'num_claims': [113, 76]}
{'Supported': [66, 47], 'Not Supported': [2, 22], 'num_claims': [72, 71]}
{'Supported': [81, 172], 'Not Supported': [23, 18], 'num_claims': [105, 203]}
{'Supported': [60, 64], 'Not Supported': [7, 5], 'num_claims': [83, 84]}
{'Supported': [118, 43], 'Not Supported': [6, 12], 'num_claims': [131, 55]}
{'Supported': [77, 30], 'Not Supported': [37, 52], 'num_claims': [115, 91]}
{'Supported': [51, 69], 'Not Supported': [5, 8], 'num_claims': [57, 80]}
{'Supported': [75, 78], 'Not Supported': [7, 6], 'num_claims': [92, 97]}
{'Supported': [82, 88], 'Not Supported': [5, 7], 'num_claims': [107, 106]}
{'Supported': [78, 57], 'Not Supported': [13, 25], 'num_claims': [95, 99]}
{'Supported': [79, 38], 'Not Supported': [15, 29], 'num_claims': [101, 68]}
{'Supported': [73, 64], 

In [30]:
files = [
    # "results/2024-05-28-00-00-55-SAFE.json",
    # "results/2024-05-27-23-57-55-SAFE.json",
    # "results/2024-05-29-01-29-26-SAFE.json",
    # "results/2024-05-29-01-30-21-SAFE.json",
    # "results/gemma2bsft-object_gen_3-SAFE-shard0.json",
    # "results/gemma2bsft-object_gen_3-SAFE-shard1.json",
    # "results/gemma2bsft-object_gen_4-SAFE-shard0.json",
    # "results/gemma2bsft-object_gen_4-SAFE-shard1.json",
    # "results/gemma2bsft-object_gen_5-SAFE-shard0.json",
    # "results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-fg-fb1.0-fw5.0-gen0523.json"
    # "results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-fg-gen0614-SAFE-shard0.json",
    # "results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-fg-gen0614-SAFE-shard1.json"
    # "results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-fg-cali0.7-gen0615-SAFE.json",
    # "results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-fg-cali0.8-gen0615-SAFE.json",
    # "results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-fg-cali0.9-gen0615-SAFE.json",
    # "results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-fg-cali0.95-gen0615-SAFE.json"
    # "results/gemma2bsft-object_38-SAFE-shard0.json",
    # "results/gemma2bsft-kto-mix-14k-lf-response-f1_0.75-fg-object_38-SAFE-shard0.json",
    "results/phi3mini-temp0.5-object_2k-SAFE.json",
    # "results/llama_3_8b-temp0.5-object_2k-SAFE.json",
    # "results/phi3mini-kto-mix-14k-lf-response-phi3-f1_100_0.7-fg1.0-kto-fg-fb0.5-object_2k-SAFE.json",
    # "results/llama_3_8b-llama3-kto-mix-14k-lf-response-llama3-f1_100_0.8-fg0.5-fgudw4.0-kto-fg-object_2k-SAFE.json"
]

lfs = []
for filename in files:
    with open(filename) as jsonfile:
        lfs.append(json.load(jsonfile))

from copy import deepcopy

longfact = deepcopy(lfs[0])

for lf in lfs[1:]:
    longfact["per_prompt_data"].extend(lf["per_prompt_data"])

print(len(longfact["per_prompt_data"]))

longfact["per_prompt_data"] = [prompt for prompt in longfact["per_prompt_data"] if "side2_posthoc_eval_data" in prompt]

print(len(longfact["per_prompt_data"]))

# Delete prompts that are in the test set
longfact["per_prompt_data"] = [prompt for prompt in longfact["per_prompt_data"] if prompt["prompt"] not in test_prompts]

print(len(longfact["per_prompt_data"]))


2330
2330
2330


In [29]:
test_prompts = set()

for f in glob.glob("longfact/longfact-objects_gpt4_01-12-2024_noduplicates/*.jsonl"):
    data = [json.loads(line.strip()) for line in open(f)]
    for d in data:
        test_prompts.add(d["prompt"])


In [4]:
def extract_entity(prompt):
    # Remove the punctuation marks from the prompt
    for s in ['.', ',', '?', '!', ':', ';', '"', "'", '(', ')', '[', ']', '{', '}', '<', '>', '/']:
        prompt = prompt.replace(s, '')
    
    # Extract the capitalized words from the prompt
    return " ".join([word.strip() for word in prompt.split()[1:] if word[0].isupper() or word.isnumeric()])

test_prompt_entities = dict()
for prompt in test_prompts:
    test_prompt_entities[extract_entity(prompt)] = prompt

In [5]:
prompts = set()

for f in glob.glob("datasets/longfact-objects_gpt4turbo_*/longfact-objects_*.jsonl"):
    data = [json.loads(line.strip()) for line in open(f)]
    for d in data:
        prompts.add(d["prompt"])

prompt_entities = dict()
for prompt in prompts:
    prompt_entities[extract_entity(prompt)] = prompt

In [22]:
# count = 0

# os.makedirs("datasets/longfact-objects_gpt4turbo_filtered", exist_ok=True)
# with open("datasets/longfact-objects_gpt4turbo_filtered/data.jsonl", "w") as jsonfile:
#     for entity, prompt in prompt_entities.items():
#         if entity in test_prompt_entities:
#             count += 1
#             continue
#         jsonfile.write(json.dumps({"prompt": prompt}) + "\n")


In [31]:
# Check for repetitions in the responses
# by checking the n-gram repetitions
def check_repetitions(response, n=10, threshold=5):
    ngram_counts = {}
    tokens = response.split()
    for i in range(len(tokens) - n):
        ngram = tuple(tokens[i:i+n])
        if ngram in ngram_counts:
            ngram_counts[ngram] += 1
        else:
            ngram_counts[ngram] = 1
    
    repetitives = []
    for ngram, count in ngram_counts.items():
        if count > threshold:
            repetitives.append((ngram, count))
    
    return repetitives

In [19]:
files = [
    # "results/2024-05-28-00-00-55-SAFE.json",
    # "results/2024-05-27-23-57-55-SAFE.json",
    # "results/2024-05-29-01-29-26-SAFE.json",
    # "results/2024-05-29-01-30-21-SAFE.json",
    # "results/gemma2bsft-object_gen_3-SAFE-shard0.json",
    # "results/gemma2bsft-object_gen_3-SAFE-shard1.json",
    # "results/gemma2bsft-object_gen_4-SAFE-shard0.json",
    # "results/gemma2bsft-object_gen_4-SAFE-shard1.json",
    # "results/gemma2bsft-object_gen_5-SAFE-shard0.json",
    "results/gemma2bsft-object_114.json"
]

files = sorted(glob.glob("results/llama*-*object_*.json"))

for f in files:
    with open(f) as jsonfile:
        lf = json.load(jsonfile)

    # lf["per_prompt_data"] = [prompt for prompt in lf["per_prompt_data"] if "side2_posthoc_eval_data" in prompt]

    # calculate the num_claims, Supported, and f1_-1 for repetition and non-repetition separately
    num_claims = [0, 0]
    Supported = [0, 0]
    f1 = [0, 0]

    repetition_counts = 0
    for prompt in lf["per_prompt_data"]:
        repetitions = check_repetitions(prompt["side2_response"], n=10, threshold=5)
        if repetitions:
            repetition_counts += 1
            if "side2_posthoc_eval_data" in prompt:
                num_claims[1] += prompt["side2_posthoc_eval_data"]["num_claims"]
                Supported[1] += prompt["side2_posthoc_eval_data"]["Supported"]
                f1[1] += prompt["side2_posthoc_eval_data"]["f1_-1"]
        else:
            if "side2_posthoc_eval_data" in prompt:
                num_claims[0] += prompt["side2_posthoc_eval_data"]["num_claims"]
                Supported[0] += prompt["side2_posthoc_eval_data"]["Supported"]
                f1[0] += prompt["side2_posthoc_eval_data"]["f1_-1"]

    print(f)
    print("\t#C\tSup\tf1")
    if "side2_avg_num_claims" in lf:
        print("\t{:.2f}\t{:.2f}\t{:.3f}".format(lf["side2_avg_num_claims"], lf["side2_avg_Supported"], lf["side2_avg_f1_-1"]))
    
    if repetition_counts > 0:
        num_claims_rep = num_claims[1] / repetition_counts
        Supported_rep = Supported[1] / repetition_counts
        f1_rep = f1[1] / repetition_counts
        print("Rep\t{:.2f}\t{:.2f}\t{:.3f}".format(num_claims_rep, Supported_rep, f1_rep))

        num_claims_nrep = num_claims[0] / (len(lf["per_prompt_data"]) - repetition_counts)
        Supported_nrep = Supported[0] / (len(lf["per_prompt_data"]) - repetition_counts)
        f1_nrep = f1[0] / (len(lf["per_prompt_data"]) - repetition_counts)
        print("Non-Rep\t{:.2f}\t{:.2f}\t{:.3f}".format(num_claims_nrep, Supported_nrep, f1_nrep))

    print(repetition_counts, "\t", len(lf["per_prompt_data"]))
    print()

results/llama_2_70b-object_114.json
	#C	Sup	f1
Rep	0.00	0.00	0.000
Non-Rep	0.00	0.00	0.000
1 	 114

results/llama_2_70b-object_38-SAFE-shard0.json
	#C	Sup	f1
	111.11	89.26	0.842
0 	 19

results/llama_2_70b-object_38-SAFE-shard1.json
	#C	Sup	f1
	113.95	89.26	0.834
0 	 19

results/llama_2_70b-object_38.json
	#C	Sup	f1
0 	 38

results/llama_3_8b-object_114.json
	#C	Sup	f1
Rep	0.00	0.00	0.000
Non-Rep	0.00	0.00	0.000
1 	 114



In [17]:
m1 = {}
for split in ["train", "test"]:
    for i, prompt in enumerate(d1[split]):
        m1[prompt["prompt"][0]["content"]] = prompt["label"]

m2 = {}
for split in ["train", "test"]:
    for i, prompt in enumerate(d2[split]):
        m2[prompt["prompt"][0]["content"]] = prompt["label"]

In [25]:
files = [
    "results/2024-05-28-00-00-55-SAFE.json",
    "results/2024-05-27-23-57-55-SAFE.json",
    "results/2024-05-29-01-29-26-SAFE.json",
    "results/2024-05-29-01-30-21-SAFE.json",
    "results/gemma2bsft-object_gen_3-SAFE-shard0.json",
    "results/gemma2bsft-object_gen_3-SAFE-shard1.json",
    "results/gemma2bsft-object_gen_4-SAFE-shard0.json",
    "results/gemma2bsft-object_gen_4-SAFE-shard1.json",
    "results/gemma2bsft-object_gen_5-SAFE-shard0.json",
]

# For each file, load the data and check if the prompts are the same as another file
# If so, print both filenames and how many prompts are the same
lfs = {}
for f in files:
    with open(f) as jsonfile:
        lf = json.load(jsonfile)

    lf["per_prompt_data"] = [prompt for prompt in lf["per_prompt_data"] if "side2_posthoc_eval_data" in prompt]
    lfs[f] = lf["per_prompt_data"]

for f, lf in lfs.items():
    for f2, lf2 in lfs.items():
        if f == f2:
            continue

        same_prompts = 0
        for prompt in lf:
            for prompt2 in lf2:
                if prompt["prompt"] == prompt2["prompt"]:
                    same_prompts += 1
                    break
        if same_prompts > 0:
            print(f, f2, same_prompts)

results/2024-05-28-00-00-55-SAFE.json results/2024-05-27-23-57-55-SAFE.json 3
results/2024-05-28-00-00-55-SAFE.json results/2024-05-29-01-29-26-SAFE.json 2
results/2024-05-28-00-00-55-SAFE.json results/2024-05-29-01-30-21-SAFE.json 25
results/2024-05-28-00-00-55-SAFE.json results/gemma2bsft-object_gen_3-SAFE-shard0.json 4
results/2024-05-28-00-00-55-SAFE.json results/gemma2bsft-object_gen_3-SAFE-shard1.json 64
results/2024-05-28-00-00-55-SAFE.json results/gemma2bsft-object_gen_4-SAFE-shard0.json 3
results/2024-05-28-00-00-55-SAFE.json results/gemma2bsft-object_gen_4-SAFE-shard1.json 74
results/2024-05-28-00-00-55-SAFE.json results/gemma2bsft-object_gen_5-SAFE-shard0.json 3
results/2024-05-27-23-57-55-SAFE.json results/2024-05-28-00-00-55-SAFE.json 3
results/2024-05-27-23-57-55-SAFE.json results/2024-05-29-01-29-26-SAFE.json 30
results/2024-05-27-23-57-55-SAFE.json results/2024-05-29-01-30-21-SAFE.json 2
results/2024-05-27-23-57-55-SAFE.json results/gemma2bsft-object_gen_3-SAFE-shard0.j

In [31]:
dir2prompt = {}

for directory in glob.glob("datasets/longfact-objects*"):
    prompts = []
    for filename in glob.glob(f"{directory}/*.jsonl"):
        with open(filename) as jsonfile:
            data = [json.loads(line.strip()) for line in jsonfile]
        prompts.extend([d["prompt"] for d in data])
    
    dir2prompt[directory] = prompts

dir2prompt_test = {}
for directory in ["longfact/object_sampled_38", "longfact/object_sampled_114", "longfact/longfact-objects_gpt4_01-12-2024_noduplicates"]:
    prompts = []
    for filename in glob.glob(f"{directory}/*.jsonl"):
        with open(filename) as jsonfile:
            data = [json.loads(line.strip()) for line in jsonfile]
        prompts.extend([d["prompt"] for d in data])
    
    dir2prompt_test[directory] = prompts


In [13]:
for i, (prompt, reps) in enumerate(zip(longfact["per_prompt_data"], repetitions)):
    if reps:
        # print(prompt["prompt"])
        print(i)
        # for sentence, atomic_facts in prompt["side2_posthoc_eval_data"]["sentences_and_atomic_facts"]:
        #     print(sentence)
        #     print(atomic_facts)
        # print("===")
        # for statement in prompt["side2_posthoc_eval_data"]["checked_statements"]:
        #     print(statement["sentence"])
        #     print(statement["atomic_fact"])
        #     print(statement["self_contained_atomic_fact"])
        #     print(statement["annotation"])

        # break

19
63
81
97
100
108
113
114
118
144
155
165
174
185
192
194
206
210
216
218
223
230
289
351
353
365
373
387
411
420
433
439
461
473
486
489
500
508
509
520
528
533
559
582
593
644
651
652
661
726
750
761
768
773
781
797
831
844
876
889
901
913
922
923
926
929
959
987
1006
1051
1084
1093
1109
1119
1134
1155
1174
1216
1227
1284
1301
1304
1323
1326
1327
1340
1351
1357
1365
1398
1408
1418
1420
1426
1439
1447
1460
1488
1499
1517
1529
1531
1533
1558
1579
1639
1648
1661
1683
1699
1701
1704
1706
1730
1771
1785
1791
1793
1806
1812
1817
1857
1863
1869
1871
1880
1897
1936
1957
1973
1975
1977
1979
1987
2010
2058
2070
2078
2080
2087
2098
2120
2150
2163
2229
2252
2272
2276
2296
2311
2333
2351
2357
2374
2375
2413
2441
2444
2471
2485
2522
2542
2548
2561


In [33]:
for i, prompt in enumerate(longfact["per_prompt_data"]):
    if len(repetitions[i]) > 0:
        print(prompt["side2_response"])
        print("---")
        print(repetitions[i])
        print("\n============\n")

The Porphyrian Tree is a concept in the esoteric and occult tradition that is often associated with the figure of Hermes Trismegistus and his teachings, which are said to embody the wisdom of the ancient Egyptian and Hebrew traditions. The Porphyrian Tree is a symbolic representation of the interconnectedness of all things and the flow of energy and information through the universe.

According to some interpretations, the Porphyrian Tree is a diagram that depicts the different levels and aspects of the divine, the divine feminine, and the divine masculine. It is said to be a map or a blueprint that shows the relationships and interactions between these different aspects of the divine.

The Porphyrian Tree is often depicted as a circular or spiral-shaped diagram, with a central point or axis that represents the divine or the center of the universe. The tree is said to be made up of multiple branches or levels, each representing a different aspect of the divine.

The names of the differe

In [33]:
#############################################
# Preparing data for factuality training
#############################################


dataset = {
    "prompt": [],
    "completion": [],
    "completion_sentences": [],
    "label": [],
    "sentence_label": []
}

K = 100
metric_type = "f1"
threshold = 0.84
sentence_precision_threshold = 0.5

sentence_precision_counts = {}

for prompt in longfact["per_prompt_data"]:
    dataset["prompt"].append([{
        "content": prompt["prompt"],
        "role": "user"
    }])

    dataset["completion"].append([{
        "content": prompt["side2_response"],
        "role": "assistant"
    }])

    if "side2_posthoc_eval_data" not in prompt:
        print("WARNING: the key `side2_posthoc_eval_data` is missing. You probably didn't evaluate the response with SAFE")
        dataset["completion_sentences"].append(None)
        dataset["label"].append(False)
        dataset["sentence_label"].append(None)
        continue
    
    num_claims = prompt["side2_posthoc_eval_data"]["num_claims"]
    if num_claims == 0:
        dataset["completion_sentences"].append(None)
        dataset["label"].append(False)
        dataset["sentence_label"].append(None)
        continue

    # Check if the completion is repetitive
    repetitions = check_repetitions(prompt["side2_response"], n=10, threshold=5)
    if len(repetitions) > 0:
        # We label the completion as dispreferred
        dataset["label"].append(False)
    else:
        # We label the completion based on its metric
        supported = prompt["side2_posthoc_eval_data"]["Supported"]
        not_supported = prompt["side2_posthoc_eval_data"]["Not Supported"]
        irrelevant = prompt["side2_posthoc_eval_data"]["Irrelevant"]

        precision = supported / num_claims
        recall = min(supported / K, 1.0)

        if supported == 0:
            f1 = 0.0
        else:
            f1 = (2 * precision * recall) / (precision + recall)

        if metric_type == "f1":
            dataset["label"].append(f1 > threshold)
        elif metric_type == "precision":
            dataset["label"].append(precision > threshold)
        elif metric_type == "recall":
            dataset["label"].append(recall > threshold)
        else:
            raise ValueError()

    """
    prompt["side2_posthoc_eval_data"]["sentences_and_atomic_facts"]: List[
        [sentence1, [atomic_facts1]],
        [sentence2, [atomic_facts2]],
        ...
    ]

    prompt["side2_posthoc_eval_data"]["checked_statements"]: List[
        {
            "sentence": sentence,
            "self_contained_atomic_fact": atomic_fact,
            "annotation": annotation
        },
        ...
    ]

    Here, we will label the sentences based on the precision of the annotations.
    We will consider a sentence as supported if the precision of the annotations is greater than a threshold.
    However, some sentences may be repetitive. In this case, we will label the repetition as dispreferred (the first occurrence will be labeled by its precision).
    """
    sentences_and_atomic_facts = prompt["side2_posthoc_eval_data"]["sentences_and_atomic_facts"]
    checked_statements = prompt["side2_posthoc_eval_data"]["checked_statements"]

    sentences = []
    sentence_labels = []
    curr_sentence_annotations = []
    curr_sentence_index = 0
    curr_atomic_fact_index = 0
    prev_sentences = set()  # To check for repetitions
    last_repetition = ""
    for statement in checked_statements:
        sentence = statement["sentence"]
        atomic_fact = statement["atomic_fact"]
        
        # Move the pointer to find the atomic fact
        while True:
            # We assume that the atomic fact is always present
            if sentences_and_atomic_facts[curr_sentence_index][0] == sentence and sentences_and_atomic_facts[curr_sentence_index][1][curr_atomic_fact_index] == atomic_fact:
                break
            curr_atomic_fact_index += 1
            if curr_atomic_fact_index >= len(sentences_and_atomic_facts[curr_sentence_index][1]):
                # Move to the next sentence
                # Also conclude the current sentence
                if sentences_and_atomic_facts[curr_sentence_index][0] in prev_sentences:
                    # We label the repetition as dispreferred
                    sentence_labels.append(False)
                    sentences.append(sentences_and_atomic_facts[curr_sentence_index][0])
                    last_repetition = sentences_and_atomic_facts[curr_sentence_index][0]
                elif len(check_repetitions(sentences_and_atomic_facts[curr_sentence_index][0], n=10, threshold=3)) > 0:
                    # repetition within the sentence
                    sentence_labels.append(False)
                    sentences.append(sentences_and_atomic_facts[curr_sentence_index][0])
                elif len(curr_sentence_annotations) > 0:
                    sentence_labels.append((sum(curr_sentence_annotations) / len(curr_sentence_annotations)) >= sentence_precision_threshold)
                    sentences.append(sentences_and_atomic_facts[curr_sentence_index][0])
                    if len(curr_sentence_annotations) not in sentence_precision_counts:
                        sentence_precision_counts[len(curr_sentence_annotations)] = [0, 0]
                    sentence_precision_counts[len(curr_sentence_annotations)][1 if sentence_labels[-1] else 0] += 1
                    
                prev_sentences.add(sentences_and_atomic_facts[curr_sentence_index][0])
                curr_sentence_annotations = []
                curr_atomic_fact_index = 0
                curr_sentence_index += 1

        curr_sentence_annotations.append(1 if statement["annotation"] == "Supported" else 0)

    # Conclude the last sentence
    # For the last sentence, it might be cut off early and doesn't match prev_sentences
    # We will check if the last sentence is a prefix of the previous sentence
    if sentences_and_atomic_facts[curr_sentence_index][0] in prev_sentences or last_repetition.startswith(sentences_and_atomic_facts[curr_sentence_index][0]):
        # We label the repetition as dispreferred
        sentence_labels.append(False)
        sentences.append(sentences_and_atomic_facts[curr_sentence_index][0])
    elif len(check_repetitions(sentences_and_atomic_facts[curr_sentence_index][0], n=10, threshold=3)) > 0:
        # repetition within the sentence
        sentence_labels.append(False)
        sentences.append(sentences_and_atomic_facts[curr_sentence_index][0])
    elif len(curr_sentence_annotations) > 0:
        sentence_labels.append((sum(curr_sentence_annotations) / len(curr_sentence_annotations)) >= sentence_precision_threshold)
        sentences.append(sentences_and_atomic_facts[curr_sentence_index][0])
        if len(curr_sentence_annotations) not in sentence_precision_counts:
            sentence_precision_counts[len(curr_sentence_annotations)] = [0, 0]
        sentence_precision_counts[len(curr_sentence_annotations)][1 if sentence_labels[-1] else 0] += 1

    dataset["completion_sentences"].append(sentences)
    dataset["sentence_label"].append(sentence_labels)
        
print(sum(dataset["label"]), len(dataset["label"]))

sent_labels = [label for l in dataset["sentence_label"] if l is not None for label in l]
print(sum(sent_labels), len(sent_labels))

1072 2330
64909 81264


In [28]:
for i in sorted(sentence_precision_counts.keys()):
    print(i, sentence_precision_counts[i], sentence_precision_counts[i][1] / sum(sentence_precision_counts[i]))

1 [5188, 5849] 0.5299447313581589
2 [1676, 11645] 0.8741836198483597
3 [3693, 12650] 0.774031695527137
4 [2091, 14357] 0.8728720817120622
5 [2034, 9791] 0.8279915433403805
6 [702, 5824] 0.8924302788844621
7 [451, 2569] 0.8506622516556291
8 [133, 1316] 0.9082125603864735
9 [75, 478] 0.864376130198915
10 [35, 236] 0.8708487084870848
11 [19, 86] 0.819047619047619
12 [2, 58] 0.9666666666666667
13 [0, 18] 1.0
14 [0, 9] 1.0
15 [0, 10] 1.0
16 [0, 7] 1.0
19 [0, 1] 1.0
20 [0, 1] 1.0
24 [0, 1] 1.0
31 [0, 1] 1.0
50 [0, 1] 1.0
51 [0, 1] 1.0


In [48]:
#############################################
# Preparing data for calibration training
# Append the calibration requirement to the
# end of the prompt
#############################################

import random


dataset = {
    "prompt": [],
    "completion": [],
    "completion_sentences": [],
    "label": [],
    "sentence_label": []
}

calibration_prompt_template = " Your response should have an expected factual accuracy of {prob}%."

K = 64
target_probs = [70, 80, 90, 95]
target_range_pct = 10
sentence_precision_threshold = 1.0
# If True, the target probability is selected based on the precision of the prompt to make sure the labels are balanced
smart_label = True

stats = {prob: [0, 0] for prob in target_probs}
precision_bins = {prob: 0 for prob in range(0, 101, 5)}

for prompt in longfact["per_prompt_data"]:
    if "factual accuracy" in prompt["prompt"]:
        # Extract the target probability from the prompt
        target_prob = None
        for prob in target_probs:
            if f"{prob}%" in prompt["prompt"]:
                target_prob = prob
                break
        assert target_prob is not None
        prompt_text = prompt["prompt"]
    else:
        # Randomly select a target probability
        target_prob = random.choice(target_probs)
    
        # Append the calibration requirement to the end of the prompt
        prompt_text = prompt["prompt"] + calibration_prompt_template.format(prob=target_prob)

    dataset["prompt"].append([{
        "content": prompt_text,
        "role": "user"
    }])

    dataset["completion"].append([{
        "content": prompt["side2_response"],
        "role": "assistant"
    }])
    
    num_claims = prompt["side2_posthoc_eval_data"]["num_claims"]
    if num_claims == 0:
        dataset["completion_sentences"].append(None)
        dataset["label"].append(False)
        dataset["sentence_label"].append(None)
        continue
    
    supported = prompt["side2_posthoc_eval_data"]["Supported"]
    not_supported = prompt["side2_posthoc_eval_data"]["Not Supported"]
    irrelevant = prompt["side2_posthoc_eval_data"]["Irrelevant"]

    precision = supported / num_claims
    recall = min(supported / K, 1.0)

    if supported == 0:
        f1 = 0.0
    else:
        f1 = (2 * precision * recall) / (precision + recall)

    # Label = 1 if the precision is within {target_range_pct}% of the target probability
    dataset["label"].append(abs(precision * 100 - target_prob) <= target_range_pct)
    stats[target_prob][int(dataset["label"][-1])] += 1

    # Find the bin for the precision
    bin = int(precision * 100 / 5) * 5
    precision_bins[bin] += 1

    sentence_to_answers = {}
    for statement in prompt["side2_posthoc_eval_data"]["checked_statements"]:
        if statement["sentence"] not in sentence_to_answers:
            sentence_to_answers[statement["sentence"]] = []
        sentence_to_answers[statement["sentence"]].append(1 if statement["annotation"] == "Supported" else 0)

    sentences = []
    sentence_labels = []
    for sentence, answers in sentence_to_answers.items():
        if len(answers) == 0:
            continue

        sentences.append(sentence)
        sentence_labels.append((sum(answers) / len(answers)) >= sentence_precision_threshold)

    # dataset["completion_sentences"].append(sentences)
    # dataset["sentence_label"].append(sentence_labels)

    dataset["completion_sentences"].append(None)
    dataset["sentence_label"].append(None)
        

In [43]:
#############################################
# Preparing data for calibration training
# Append the calibration requirement to the
# end of the prompt.
# This block asks the model for a factual
# precision of {prob}% or higher.
#############################################

import random


dataset = {
    "prompt": [],
    "completion": [],
    "completion_sentences": [],
    "label": [],
    "sentence_label": []
}

calibration_prompt_template = " Your response should have a factual precision of {prob}% or higher."

K = 64
target_probs = [70, 75, 80, 85, 90, 95]
sentence_precision_threshold = 1.0

stats = {prob: [0, 0] for prob in target_probs}
precision_bins = {prob: [] for prob in range(0, 101, 5)}

for prompt in longfact["per_prompt_data"]:
    if "factual precision" in prompt["prompt"]:
        # Extract the target probability from the prompt
        target_prob = None
        for prob in target_probs:
            if f"{prob}%" in prompt["prompt"]:
                target_prob = prob
                break
        assert target_prob is not None
        prompt_text = prompt["prompt"]
    else:
        # Randomly select a target probability
        target_prob = random.choice(target_probs)
    
        # Append the calibration requirement to the end of the prompt
        prompt_text = prompt["prompt"] + calibration_prompt_template.format(prob=target_prob)

    dataset["prompt"].append([{
        "content": prompt_text,
        "role": "user"
    }])

    dataset["completion"].append([{
        "content": prompt["side2_response"],
        "role": "assistant"
    }])
    
    num_claims = prompt["side2_posthoc_eval_data"]["num_claims"]
    if num_claims == 0:
        dataset["completion_sentences"].append(None)
        dataset["label"].append(False)
        dataset["sentence_label"].append(None)
        continue
    
    supported = prompt["side2_posthoc_eval_data"]["Supported"]
    not_supported = prompt["side2_posthoc_eval_data"]["Not Supported"]
    irrelevant = prompt["side2_posthoc_eval_data"]["Irrelevant"]

    precision = supported / num_claims
    recall = min(supported / K, 1.0)

    if supported == 0:
        f1 = 0.0
    else:
        f1 = (2 * precision * recall) / (precision + recall)

    if precision >= 0.95:
        target_prob = 95
        prompt_text = prompt["prompt"] + calibration_prompt_template.format(prob=target_prob)
        dataset["prompt"][-1] = [{
            "content": prompt_text,
            "role": "user"
        }]
    elif precision >= 0.9:
        target_prob = 90
        prompt_text = prompt["prompt"] + calibration_prompt_template.format(prob=target_prob)
        dataset["prompt"][-1] = [{
            "content": prompt_text,
            "role": "user"
        }]
    elif precision >= 0.85 and random.random() > 0.5:
        target_prob = 85
        prompt_text = prompt["prompt"] + calibration_prompt_template.format(prob=target_prob)
        dataset["prompt"][-1] = [{
            "content": prompt_text,
            "role": "user"
        }]
    elif precision >= 0.8 and random.random() > 0.5:
        target_prob = 80
        prompt_text = prompt["prompt"] + calibration_prompt_template.format(prob=target_prob)
        dataset["prompt"][-1] = [{
            "content": prompt_text,
            "role": "user"
        }]
        
    # Label = 1 if the precision is equal to or higher than the target probability
    dataset["label"].append(precision * 100 >= target_prob)
    stats[target_prob][int(dataset["label"][-1])] += 1

    # Find the bin for the precision
    bin = int(precision * 100 / 5) * 5
    precision_bins[bin].append(num_claims)

    # sentence_to_answers = {}
    # for statement in prompt["side2_posthoc_eval_data"]["checked_statements"]:
    #     if statement["sentence"] not in sentence_to_answers:
    #         sentence_to_answers[statement["sentence"]] = []
    #     sentence_to_answers[statement["sentence"]].append(1 if statement["annotation"] == "Supported" else 0)

    # sentences = []
    # sentence_labels = []
    # for sentence, answers in sentence_to_answers.items():
    #     if len(answers) == 0:
    #         continue

    #     sentences.append(sentence)
    #     sentence_labels.append((sum(answers) / len(answers)) >= sentence_precision_threshold)

    # dataset["completion_sentences"].append(sentences)
    # dataset["sentence_label"].append(sentence_labels)

    dataset["completion_sentences"].append(None)
    dataset["sentence_label"].append(None)
        

In [39]:
#############################################
# Preparing data for calibration training
# Append the calibration requirement to the
# end of the prompt.
# This block asks the model for a factual
# accuracy of {prob}% or higher.
# Also samples multiple target probabilities
#############################################

import random


dataset = {
    "prompt": [],
    "completion": [],
    "completion_sentences": [],
    "label": [],
    "sentence_label": []
}

calibration_prompt_template = " Your response should have a factual accuracy of {prob}% or higher."

target_probs = [60, 65, 70, 75, 80, 85, 90, 95]

stats = {prob: [0, 0] for prob in target_probs}
precision_bins = {prob: [] for prob in range(0, 101, 5)}

for prompt in longfact["per_prompt_data"]:
    num_claims = prompt["side2_posthoc_eval_data"]["num_claims"]
    if num_claims == 0:
        dataset["prompt"].append([{
            "content": prompt_text,
            "role": "user"
        }])

        dataset["completion"].append([{
            "content": prompt["side2_response"],
            "role": "assistant"
        }])
        
        dataset["completion_sentences"].append(None)
        dataset["label"].append(False)
        dataset["sentence_label"].append(None)
        continue

    supported = prompt["side2_posthoc_eval_data"]["Supported"]
    not_supported = prompt["side2_posthoc_eval_data"]["Not Supported"]
    irrelevant = prompt["side2_posthoc_eval_data"]["Irrelevant"]

    precision = supported / num_claims

    # Find the bin for the precision
    bin = int(precision * 100 / 5) * 5
    precision_bins[bin].append(num_claims)

    sampled_target_probs = set(random.choices(target_probs, k=3))

    if precision >= 0.95:
        sampled_target_probs.add(95)
    elif precision >= 0.9:
        sampled_target_probs.add(90)
    elif precision >= 0.85 and random.random() > 0.5:
        sampled_target_probs.add(85)
    elif precision >= 0.8 and random.random() > 0.5:
        sampled_target_probs.add(80)

    for target_prob in sampled_target_probs:
        # Append the calibration requirement to the end of the prompt
        prompt_text = prompt["prompt"] + calibration_prompt_template.format(prob=target_prob)

        dataset["prompt"].append([{
            "content": prompt_text,
            "role": "user"
        }])

        dataset["completion"].append([{
            "content": prompt["side2_response"],
            "role": "assistant"
        }])
              
        # Label = 1 if the precision is equal to or higher than the target probability
        dataset["label"].append(precision * 100 >= target_prob)
        stats[target_prob][int(dataset["label"][-1])] += 1

        dataset["completion_sentences"].append(None)
        dataset["sentence_label"].append(None)
            

In [49]:
stats

{70: [349, 232], 80: [193, 399], 90: [298, 258], 95: [431, 170]}

In [50]:
dataset["prompt"][-3]

[{'content': 'What occurred at the Chernobyl Nuclear Power Plant in 1986? Provide as many specific details and examples as possible (such as names of people, numbers, events, locations, dates, times, etc.) Your response should have an expected factual accuracy of 95%.',
  'role': 'user'}]

In [51]:
for prec, l in precision_bins.items():
    print(prec, len(l), sum(l) / len(l) if len(l) > 0 else 0.0)

TypeError: object of type 'int' has no len()

In [52]:
print(sum(dataset["label"]), len(dataset["label"]))

sent_labels = [label for l in dataset["sentence_label"] if l is not None for label in l]
print(sum(sent_labels), len(sent_labels))

1059 2330
0 0


In [53]:
import random
from datasets import Dataset, DatasetDict

# Shuffle the dataset
random.seed(42)
indices = list(range(len(dataset["prompt"])))
random.shuffle(indices)
dataset = {
    "prompt": [dataset["prompt"][i] for i in indices],
    "completion": [dataset["completion"][i] for i in indices],
    "completion_sentences": [dataset["completion_sentences"][i] for i in indices],
    "label": [dataset["label"][i] for i in indices],
    "sentence_label": [dataset["sentence_label"][i] for i in indices],
}

# Split the dataset into training and test sets
train_ratio = 0.9
train_size = int(len(dataset["prompt"]) * train_ratio)
train_dataset = Dataset.from_dict({
    "prompt": dataset["prompt"][:train_size],
    "completion": dataset["completion"][:train_size],
    "completion_sentences": dataset["completion_sentences"][:train_size],
    "label": dataset["label"][:train_size],
    "sentence_label": dataset["sentence_label"][:train_size],
})
test_dataset = Dataset.from_dict({
    "prompt": dataset["prompt"][train_size:],
    "completion": dataset["completion"][train_size:],
    "completion_sentences": dataset["completion_sentences"][train_size:],
    "label": dataset["label"][train_size:],
    "sentence_label": dataset["sentence_label"][train_size:],
})

hf_dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

In [54]:
hf_dataset.push_to_hub("chaoweihuang/lf-response-phi3-iter2-calibration", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chaoweihuang/lf-response-phi3-iter2-calibration/commit/203725295e8a7e1c52af98962f135276f81a433c', commit_message='Upload dataset', commit_description='', oid='203725295e8a7e1c52af98962f135276f81a433c', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
from datasets import load_dataset

dataset = load_dataset("chaoweihuang/lf-response-f1_0.75-fg-calibration_r12pct")
# dataset = load_dataset("chaoweihuang/lf-response-f1_0.75")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [38]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

chosen_lengths = []
rejected_lengths = []

for i, inst in enumerate(train_dataset):
    if inst["label"]:
        chosen_lengths.append(len(inst["completion"][0]["content"].split()))
    else:
        rejected_lengths.append(len(inst["completion"][0]["content"].split()))

print(sum(chosen_lengths) / len(chosen_lengths))
print(sum(rejected_lengths) / len(rejected_lengths))

chosen_lengths = []
rejected_lengths = []

for i, inst in enumerate(train_dataset):
    for sent, label in zip(inst["completion_sentences"], inst["sentence_label"]):
        if label:
            chosen_lengths.append(len(sent.split()))
        else:
            rejected_lengths.append(len(sent.split()))

print(sum(chosen_lengths) / len(chosen_lengths))
print(sum(rejected_lengths) / len(rejected_lengths))

581.3459119496855
470.8821138211382
19.060908828898068
21.604860486048604


In [3]:
dataset1 = load_dataset("chaoweihuang/lf-response-phi3-f1_100_0.7-fg0.5")
dataset2 = load_dataset("chaoweihuang/lf-response-phi3-iter2-calibration")
dataset3 = load_dataset("chaoweihuang/lf-response-phi3-iter2-f1_100_0.8-fg0.5")


In [43]:
postamble = " Provide as many specific details and examples as possible (such as names of people, numbers, events, locations, dates, times, etc.)"
# postamble = ""

prompts1 = set()
for inst in dataset1["train"]:
    prompts1.add(inst["prompt"][0]["content"].replace(postamble, ""))

prompts2 = set()
for inst in dataset2["train"]:
    prompts2.add(inst["prompt"][0]["content"].replace(postamble, ""))

prompts3 = set()
for inst in dataset3["train"]:
    prompts3.add(inst["prompt"][0]["content"].replace(postamble, ""))

print(len(prompts1), len(prompts2), len(prompts3))
print(len(prompts1 & prompts2), len(prompts1 & prompts3), len(prompts2 & prompts3))

# Check overlap with test_prompts
test_prompts = set()

for f in glob.glob("longfact/object_sampled_114/*.jsonl"):
    data = [json.loads(line.strip()) for line in open(f)]
    for d in data:
        test_prompts.add(d["prompt"])

print(len(test_prompts))
print(len(prompts1 & test_prompts), len(prompts2 & test_prompts), len(prompts3 & test_prompts))


2097 2097 2097
0 2097 0
109
0 0 0


In [44]:
def extract_entity(prompt):
    # Remove the punctuation marks from the prompt
    for s in ['.', ',', '?', '!', ':', ';', '"', "'", '(', ')', '[', ']', '{', '}', '<', '>', '/']:
        prompt = prompt.replace(s, '')
    
    # Extract the capitalized words from the prompt
    return " ".join([word.strip() for word in prompt.split()[1:] if word[0].isupper() or word.isnumeric()])

test_prompt_entities = set()
for prompt in test_prompts:
    test_prompt_entities.add(extract_entity(prompt))

prompt_entities1 = set()
for prompt in prompts1:
    prompt_entities1.add(extract_entity(prompt))

prompt_entities2 = set()
for prompt in prompts2:
    prompt_entities2.add(extract_entity(prompt))

prompt_entities3 = set()
for prompt in prompts3:
    prompt_entities3.add(extract_entity(prompt))

print(len(prompt_entities1), len(prompt_entities2), len(prompt_entities3))
print(len(prompt_entities1 & prompt_entities2), len(prompt_entities1 & prompt_entities3), len(prompt_entities2 & prompt_entities3))
print(len(prompt_entities1 & test_prompt_entities), len(prompt_entities2 & test_prompt_entities), len(prompt_entities3 & test_prompt_entities))

2097 2097 2097
0 2097 0
0 0 0


In [45]:
prompts1

{'What artifacts were recovered from Pompeii?',
 'What is the Sydney Opera House?',
 'What is the Thin Film Bulk Acoustic Resonator (FBAR)?',
 'What is the Hart-Scott-Rodino Antitrust Improvements Act?',
 "What are the characteristics of the Barnard's Star?",
 'What is the Human Immunodeficiency Virus (HIV)?',
 'Who was Zhang Qian?',
 'What is the ResNet-50 model?',
 "What is the Cooper's Hill Cheese-Rolling and Wake?",
 'Who is Rina Sawayama?',
 'What is the Whitewater Arroyo virus?',
 'What is the American Institute of Certified Public Accountants (AICPA)?',
 'What is the Porphyrian Tree?',
 'What is the Venturi Mask?',
 'What was the diet of Neanderthals?',
 'What is the Triple Entry Accounting system?',
 'What is the Certified Public Accountant (CPA) Exam?',
 'Who is Hideki Matsuyama in the world of golf?',
 'What is the Shakuhachi?',
 "What is the function of the Pantheon's oculus?",
 'What is the Gallium Nitride (GaN) Power Device?',
 'What is the Treaty on the Prohibition of Nuc

In [24]:
import json

with open("debug.jsonl") as jsonfile:
    data = [json.loads(line.strip()) for line in jsonfile]

# Calculate the number of tokens for prompts and responses
prompt_lengths = []
response_lengths = []
for inst in data:
    prompt_lengths.append(len(inst["prompt"].split()))
    response_lengths.append(len(inst["response"].split()))

# Print total tokens
print(sum(prompt_lengths))
print(sum(response_lengths))

# Print total costs.
# Prompt cost = 0.15 / 1M tokens
# Response cost = 0.5 / 1M tokens
print(sum(prompt_lengths) * 0.15 / 1e6)
print(sum(response_lengths) * 0.5 / 1e6)

# Print average tokens
print(sum(prompt_lengths) / len(prompt_lengths))
print(sum(response_lengths) / len(response_lengths))

276983
41977
0.04154745
0.0209885
638.2096774193549
96.72119815668202


In [25]:
# Classify the prompts based on the task
task_count = {}
task_token_count = {}

for inst in data:
    if "Your task is to break the sentence down into a list of atomic facts" in inst["prompt"]:
        task = "atomic"
    elif "The following STATEMENT has been extracted from the broader context of the given RESPONSE to the given QUESTION." in inst["prompt"]:
        task = "relevance"
    elif "Vague references include but are not limited to" in inst["prompt"]:
        task = "rewrite"
    elif "Your goal is to try to find evidence that either supports or does not support" in inst["prompt"]:
        task = "search"
    elif "Determine whether the given STATEMENT is supported by the given KNOWLEDGE." in inst["prompt"]:
        task = "final"
    else:
        print(inst["prompt"])
        task = "other"

    if task not in task_count:
        task_count[task] = 0
    
    task_count[task] += 1

    if task not in task_token_count:
        task_token_count[task] = [0, 0]
    
    task_token_count[task][0] += len(inst["prompt"].split())
    task_token_count[task][1] += len(inst["response"].split())

print(task_count)
print(task_token_count)


{'atomic': 27, 'rewrite': 95, 'relevance': 95, 'search': 144, 'final': 73}
{'atomic': [21024, 889], 'rewrite': [74609, 1259], 'relevance': [79634, 14871], 'search': [41417, 7847], 'final': [60299, 17111]}


In [13]:
data[-1]

{'prompt': 'Instructions:\n1. You have been given a STATEMENT and some KNOWLEDGE points.\n2. Determine whether the given STATEMENT is supported by the given KNOWLEDGE. The STATEMENT does not need to be explicitly supported by the KNOWLEDGE, but should be strongly implied by the KNOWLEDGE.\n3. Before showing your answer, think step-by-step and show your specific reasoning. As part of your reasoning, summarize the main points of the KNOWLEDGE related to the STATEMENT.\n4. If the STATEMENT is supported by the KNOWLEDGE, be sure to show the supporting evidence.\n5. After stating your reasoning, restate the STATEMENT and then determine your final answer based on your reasoning and the STATEMENT.\n6. Your final answer should be either "Supported" or "Not Supported". Wrap your final answer in square brackets.\n\nKNOWLEDGE:\nDiscovery of nuclear fission | 1978, Hahn\'s narrative of the discovery of fission began to crumble. Hahn and Meitner had died in 1968, but Strassmann was still alive, and